In [1]:
import os
import pandas as pd
import numpy as np
import math
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib import cm
from parse import *
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier as RandomForest
import random
from sklearn import linear_model
from datetime import datetime
import gc

pd.set_option("display.max_columns", 2000)


In [2]:
CHUNK_SIZE = 100000

start = datetime.now()

reader_date = pd.read_csv("train_date.csv",chunksize=CHUNK_SIZE, dtype=np.float32)
reader_num = pd.read_csv("output/df_id_response.csv",chunksize=CHUNK_SIZE, dtype=np.float32)

#分割して読み込み
loop = 0
cnt_0 = 0
cnt_1 = 0
for df_chunk_date in reader_date:
    df_chunk_num = reader_num.get_chunk(CHUNK_SIZE)
    
    df_chunk = pd.merge(on="Id", left=df_chunk_date, right=df_chunk_num)
    cnt_0 = cnt_0 + len(df_chunk[df_chunk['Response'] == 0].index)
    cnt_1 = cnt_1 + len(df_chunk[df_chunk['Response'] == 1].index)

    if loop == 0:
        df = df_chunk
    else:
        df = pd.concat([df, df_chunk])
    
    print("loop : {} / Rows = {}".format(loop, len(df.index)))

    loop = loop + 1

print("0:{}, 1:{}".format(cnt_0, cnt_1))
print("{} -> {}".format(start, datetime.now()))
print("Memory:{}".format(df.memory_usage(index=True).sum()))

loop : 0 / Rows = 100000
loop : 1 / Rows = 200000
loop : 2 / Rows = 300000
loop : 3 / Rows = 400000
loop : 4 / Rows = 500000
loop : 5 / Rows = 600000
loop : 6 / Rows = 700000
loop : 7 / Rows = 800000
loop : 8 / Rows = 900000
loop : 9 / Rows = 1000000
loop : 10 / Rows = 1100000
loop : 11 / Rows = 1183747
0:1176868, 1:6879
2018-07-18 22:03:01.009947 -> 2018-07-18 22:04:52.034437
Memory:5492586080


In [3]:
df.tail()

Id     L0_S0_D1     L0_S0_D3     L0_S0_D5     L0_S0_D7  \
83742  2367490.0  1239.900024  1239.900024  1239.900024  1239.900024   
83743  2367491.0          NaN          NaN          NaN          NaN   
83744  2367492.0  1482.180054  1482.180054  1482.180054  1482.180054   
83745  2367493.0   608.830017   608.830017   608.830017   608.830017   
83746  2367495.0   937.609985   937.609985   937.609985   937.609985   

          L0_S0_D9    L0_S0_D11    L0_S0_D13    L0_S0_D15    L0_S0_D17  \
83742  1239.900024  1239.900024  1239.900024  1239.900024  1239.900024   
83743          NaN          NaN          NaN          NaN          NaN   
83744  1482.180054  1482.180054  1482.180054  1482.180054  1482.180054   
83745   608.830017   608.830017   608.830017   608.830017   608.830017   
83746   937.609985   937.609985   937.609985   937.609985   937.609985   

         L0_S0_D19    L0_S0_D21    L0_S0_D23    L0_S1_D26    L0_S1_D30  \
83742  1239.900024  1239.900024  1239.900024  1239.900024  1239.900024   
83743          NaN          NaN          NaN          NaN          NaN   
83744  1482.180054  1482.180054  1482.180054  1482.180054  1482.180054   
83745   608.830017   608.830017   608.830017   608.830017   608.830017   
83746   937.609985   937.609985   937.609985   937.609985   937.609985   

         L0_S2_D34    L0_S2_D38    L0_S2_D42    L0_S2_D46    L0_S2_D50  \
83742  1239.910034  1239.910034  1239.910034  1239.910034  1239.910034   
83743          NaN          NaN          NaN          NaN          NaN   
83744          NaN          NaN          NaN          NaN          NaN   
83745          NaN          NaN          NaN          NaN          NaN   
83746   937.609985   937.609985   937.609985   937.609985   937.609985   

         L0_S2_D54    L0_S2_D58    L0_S2_D62    L0_S2_D66    L0_S3_D70  \
83742  1239.910034  1239.910034  1239.910034  1239.910034          NaN   
83743          NaN          NaN          NaN          NaN          NaN   
83744          NaN          NaN          NaN          NaN  1482.189941   
83745          NaN          NaN          NaN          NaN   608.830017   
83746   937.609985   937.609985   937.609985   937.609985          NaN   

         L0_S3_D74    L0_S3_D78    L0_S3_D82    L0_S3_D86    L0_S3_D90  \
83742          NaN          NaN          NaN          NaN          NaN   
83743          NaN          NaN          NaN          NaN          NaN   
83744  1482.189941  1482.189941  1482.189941  1482.189941  1482.189941   
83745   608.830017   608.830017   608.830017   608.830017   608.830017   
83746          NaN          NaN          NaN          NaN          NaN   

         L0_S3_D94    L0_S3_D98   L0_S3_D102   L0_S4_D106   L0_S4_D111  \
83742          NaN          NaN          NaN  1239.930054  1239.930054   
83743          NaN          NaN          NaN          NaN          NaN   
83744  1482.189941  1482.189941  1482.189941          NaN          NaN   
83745   608.830017   608.830017   608.830017   608.849976   608.849976   
83746          NaN          NaN          NaN   937.640015   937.640015   

        L0_S5_D115   L0_S5_D117   L0_S6_D120   L0_S6_D124   L0_S6_D127  \
83742          NaN          NaN  1239.930054  1239.930054  1239.930054   
83743          NaN          NaN          NaN          NaN          NaN   
83744  1482.199951  1482.199951  1482.209961  1482.209961  1482.209961   
83745          NaN          NaN          NaN          NaN          NaN   
83746          NaN          NaN          NaN          NaN          NaN   

        L0_S6_D130   L0_S6_D134  L0_S7_D137  L0_S7_D139  L0_S7_D140  \
83742  1239.930054  1239.930054         NaN         NaN         NaN   
83743          NaN          NaN         NaN         NaN         NaN   
83744  1482.209961  1482.209961         NaN         NaN         NaN   
83745          NaN          NaN  608.849976  608.849976  608.849976   
83746          NaN          NaN  937.640015  937.640015  937.640015   

       L0_S7_D141  L0_S7_D143   L0_S8_D

In [5]:
df_date_ok = df.loc[df['Response'] == 0]
df_date_ng = df.loc[df['Response'] == 1]

In [6]:
def classify_in_range(row, max_col, min_col):
    if ((row <= max_col) and (min_col <= row)):
        return 0
    if ((row > max_col) or (min_col > row)):
        return 1
    return np.nan



In [20]:
date_columns = ['Id', 'L1_S25_D2713', 'L1_S25_D2518', 'L1_S25_D2505', 'L1_S25_D2497', 'L1_S24_D1368']

In [23]:
df_tmp_cnv = df[date_columns]
for col in date_columns:
    if (col != 'Id'):
        max_col = df_date_ng[col].max()
        min_col = df_date_ng[col].min()
        df_tmp_cnv[col] = df[col].apply(lambda row:classify_in_range(row, max_col, min_col))



C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [22]:
col

'Id'

In [29]:
df_tmp_cnv.L1_S24_D1368.unique()

array([nan,  0.,  1.])

In [25]:
df.head(10)

Id     L0_S0_D1     L0_S0_D3     L0_S0_D5     L0_S0_D7     L0_S0_D9  \
0   4.0    82.239998    82.239998    82.239998    82.239998    82.239998   
1   6.0          NaN          NaN          NaN          NaN          NaN   
2   7.0  1618.699951  1618.699951  1618.699951  1618.699951  1618.699951   
3   9.0  1149.199951  1149.199951  1149.199951  1149.199951  1149.199951   
4  11.0   602.640015   602.640015   602.640015   602.640015   602.640015   
5  13.0  1331.660034  1331.660034  1331.660034  1331.660034  1331.660034   
6  14.0          NaN          NaN          NaN          NaN          NaN   
7  16.0          NaN          NaN          NaN          NaN          NaN   
8  18.0   517.640015   517.640015   517.640015   517.640015   517.640015   
9  23.0          NaN          NaN          NaN          NaN          NaN   

     L0_S0_D11    L0_S0_D13    L0_S0_D15    L0_S0_D17    L0_S0_D19  \
0    82.239998    82.239998    82.239998    82.239998    82.239998   
1          NaN          NaN          NaN          NaN          NaN   
2  1618.699951  1618.699951  1618.699951  1618.699951  1618.699951   
3  1149.199951  1149.199951  1149.199951  1149.199951  1149.199951   
4   602.640015   602.640015   602.640015   602.640015   602.640015   
5  1331.660034  1331.660034  1331.660034  1331.660034  1331.660034   
6          NaN          NaN          NaN          NaN          NaN   
7          NaN          NaN          NaN          NaN          NaN   
8   517.640015   517.640015   517.640015   517.640015   517.640015   
9          NaN          NaN          NaN          NaN          NaN   

     L0_S0_D21    L0_S0_D23    L0_S1_D26    L0_S1_D30    L0_S2_D34  \
0    82.239998    82.239998    82.239998    82.239998    82.239998   
1          NaN          NaN          NaN          NaN          NaN   
2  1618.699951  1618.699951  1618.699951  1618.699951  1618.699951   
3  1149.199951  1149.199951  1149.199951  1149.199951  1149.209961   
4   602.640015   602.640015   602.640015   602.640015          NaN   
5  1331.660034  1331.660034  1331.660034  1331.660034          NaN   
6          NaN          NaN          NaN          NaN          NaN   
7          NaN          NaN          NaN          NaN          NaN   
8   517.640015   517.640015   517.640015   517.640015   517.640015   
9          NaN          NaN          NaN          NaN          NaN   

     L0_S2_D38    L0_S2_D42    L0_S2_D46    L0_S2_D50    L0_S2_D54  \
0    82.239998    82.239998    82.239998    82.239998    82.239998   
1          NaN          NaN          NaN          NaN          NaN   
2  1618.699951  1618.699951  1618.699951  1618.699951  1618.699951   
3  1149.209961  1149.209961  1149.209961  1149.209961  1149.209961   
4          NaN          NaN          NaN          NaN          NaN   
5          NaN          NaN          NaN          NaN          NaN   
6          NaN          NaN          NaN          NaN          NaN   
7          NaN          NaN          NaN          NaN          NaN   
8   517.640015   517.640015   517.640015   517.640015   517.640015   
9          NaN          NaN          NaN          NaN          NaN   

     L0_S2_D58    L0_S2_D62    L0_S2_D66    L0_S3_D70    L0_S3_D74  \
0    82.239998    82.239998    82.239998          NaN          NaN   
1          NaN          NaN          NaN          NaN          NaN   
2  1618.699951  1618.699951  1618.699951          NaN          NaN   
3  1149.209961  1149.209961  1149.209961          NaN          NaN   
4          NaN          NaN          NaN   602.640015   602.640015   
5          NaN          NaN          NaN  1331.670044  1331.670044   
6          NaN          NaN          NaN          NaN          NaN   
7          NaN          NaN          NaN          NaN          NaN   
8   517.640015   517.640015   517.640015          NaN          NaN   
9          NaN          NaN          NaN          NaN          NaN   

     L0_S3_D78    L0_S3_D82    L0_S3_D86    L0_S3_D90    L0_S3_D94  \
0          NaN

In [31]:
df_tmp_cnv.to_csv('output/train_date_top5.csv', index=False)

In [37]:
df_test = pd.read_csv("test_date.csv", dtype=np.float32)

In [38]:
df_tmp_test_cnv = df_test[date_columns]
for col in date_columns:
    if (col != 'Id'):
        max_col = df_date_ng[col].max()
        min_col = df_date_ng[col].min()
        df_tmp_test_cnv[col] = df_test[col].apply(lambda row:classify_in_range(row, max_col, min_col))


C:\Users\Takumi\Anaconda3\envs\Kaggle\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
df_tmp_test_cnv.to_csv('output/test_date_top5.csv', index=False)

In [40]:
df_tmp_test_cnv.head()

,Id,L1_S25_D2713,L1_S25_D2518,L1_S25_D2505,L1_S25_D2497,L1_S24_D1368
0,1.0,NaN,NaN,NaN,NaN,NaN
1,2.0,NaN,NaN,NaN,NaN,NaN
2,3.0,NaN,NaN,NaN,NaN,NaN
3,5.0,NaN,NaN,NaN,NaN,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN
